# Automation Toolbox for Machine learning in water Networks

This notebook showcases all features of `atmn` (Automation Toolbox for Machine learning in water Networks). 

First, we will show you how to generate a dataset using the `atmn-generate` tool. Then we use the `atmn.ScenarioLoader` API to easily load different data from our dataset. Next, we demonstrate how `atmn-export` can be used to export data to Excel files. Lastly, we show you how to use the `atmn-visualize` tool, which can be useful for editing dataset configurations.

## Required packages
First, let's make sure `atmn` is installed in your current python environment. We will also use pandas to handle data in this demo, so it is also installed.

In [ ]:
# Install atmn
!pip install atmn

# Install and import pandas
!pip install pandas
import pandas as pd

## Generate Data
An example configuration containing several leak and sensor fault configurations is provided in `my_collection/configuration.xml`. It uses the water network description of the Hanoi network from `networks/Hanoi.inp`.

In order to simulate the scenarios, use the `atmn-generate` command. For our purposes, we will provide the `-p` flag to run simulations in parallel. If you add new scenarios or leak configurations, the generator will only simulate the scenarios, which are not already present. If you decide to change the configuration, make sure to use the `-f` flag in order to re-generate already existing data.


You can also use `atmn-generate -h` to view all options the generation tool offers.

In case your package manager did not create the `atmn` wrappers, you can always use `python -m atmn` instead.

In [ ]:
# Generate the dataset
!atmn-generate my_collection/config.xml -p

## Using the Scenario Loader
`atmn` offers the ScenarioCollection, a convenient interface to load the scenarios you just generated. For this you just need to use a single import. 

In [ ]:
from atmn import ScenarioCollection

The `ScenarioCollection` is initialized using the path to the collection. Once created, you can use it to list all available Scenarios within the collection.

In [ ]:
# Create a ScenarioCollection for the data we just generated
my_collection = ScenarioCollection('my_collection')

# List available Scenarios within the collection
print('Available Scenarios:', my_collection.list_scenarios())

Next, you can retrieve a `Scenario` from the `ScenarioCollection` using the `get_scenario` method. The `Scenario` itself exposes which configurations it offers through the `list_configs` method. You can also query information on Leaks and Sensorfaults using the `get_leak_data` and `get_sensorfault_data` methods as shown below.

In [ ]:
# Retrieve Scenario from ScenarioCollection
my_scenario = my_collection.get_scenario('Hanoi_One_Week')

# Get the available configurations from the Scenario
print('#' * 5, 'Available Configurations', '#' * 5)
configs = my_scenario.list_configs()
for config_type, cfgs in configs.items():
    print(f'{config_type}: {cfgs}')

print()

# Get information about a specific Leak config
print('#' * 5, 'Info for Leak config "MultipleLeaks"', '#' * 5)
leaks = my_scenario.get_leak_data('MultipleLeaks')
for i, leak in enumerate(leaks):
    print(f'Leak {i}: {leak}')
    
print()

# Get information about a specific Sensorfault config
print('#' * 5, 'Info for Sensorfault config "PressureAndFlow"', '#' * 5)
sensorfaults = my_scenario.get_sensorfault_data('PressureAndFlow')
for i, sensorfault in enumerate(sensorfaults):
    print(f'Sensorfault {i}: {sensorfault}')

Finally, to retrieve the actual sensor data from the `Scenario`, you can use the `get` method. It requires you to choose one configuration of each type: Leak, Sensor and Sensorfault. The loader will then combine the information from these configurations and return the sensor data.

In [ ]:
# Retrieve data from the Scenario
data = my_scenario.get('MultipleLeaks', 'Sensors1', 'PressureAndFlow')
demand = data['demand']
flow = data['flow']
pressure = data['pressure']

print('#' * 5, 'Pressures loaded from the Scenario', '#' * 5)
pressure

If you don't want to go the extra step through the `Scenario` object, you can also query all information directly from the `ScenarioCollection` using the same methods. You just have to specify the Scenario name in the first argument. For Example:

In [ ]:
# Get the available configurations for the Scenario "Hanoi_One_Month"
print('#' * 5, 'Available Configurations (Hanoi_One_Month)', '#' * 5)
configs = my_collection.list_configs('Hanoi_One_Month')
for config_type, cfgs in configs.items():
    print(f'{config_type}: {cfgs}')

# Retrieve data from the Collection directly
data = my_collection.get('Hanoi_One_Month', 'MultipleLeaks', 'Sensors1', 'PressureAndFlow')
demand = data['demand']
flow = data['flow']
pressure = data['pressure']

print()

print('#' * 5, 'Pressures directly from the Collection', '#' * 5)
pressure

## Usage Examples
Below you can find some examples where plots are generated from the data using the `ScenarioCollection`

### Plotting pressure Sensor 10 in different Scenario configurations

In [ ]:
# Create a dataframe for the plots
df = pd.DataFrame()

# Include a non leaky scenario with a sensorfault
pressure = my_scenario.get('Baseline', 'Sensors1', 'JustPressure')['pressure']['10']
df['No Leak, but Sensorfault'] = pressure

# List leak configs
leak_configs = my_scenario.list_configs()['LeakConfigs']
leak_configs.remove('MultipleLeaks') # Remove this, so we have comparable leaks

# Include different leaky scenarios without sensorfault
# Also include the baseline scenario
for leak_config in leak_configs:
    pressure = my_scenario.get(leak_config, 'Sensors1', 'GT')['pressure']['10']
    df[leak_config.replace('Pipe11', 'Leak: ')] = pressure

# Plot the dataframe
df.plot(title = 'Pressures at Node 10 over time', xlabel = 'Time [s]', ylabel = 'Pressure [m]')

### Plotting flow Sensor 1 for different leak configs, and with a sensorfault

In [ ]:
# Create a dataframe for the plots
df = pd.DataFrame()

# Include a non leaky scenario with a sensorfault
flow = my_scenario.get('Baseline', 'Sensors1', 'JustFlow')['flow']['1']
df['No Leak, but Sensorfault'] = flow

# List leak configs
leak_configs = my_scenario.list_configs()['LeakConfigs']
leak_configs = list(filter(lambda cfg: not 'Incipient' in cfg, leak_configs)) # Drop some to make plot less crowded

# Include different leaky scenarios without sensorfault
# Also include the baseline scenario
for leak_config in leak_configs:
    flow = my_scenario.get(leak_config, 'Sensors1', 'GT')['flow']['1']
    df[leak_config.replace('Pipe11', 'Leak: ')] = flow

# Plot the dataframe
df.plot(title = 'Flows through Pipe 1 over time', xlabel = 'Time [s]', ylabel = 'Flow [m^3/s]')

## Exporting
In case you want to use your simulated data in external programs, you may need to export your Scenarioto Excel format. To do so, use the `atmn-export` tool as shown below. If you need more detailed help on how to use the exporter, use the `-h` flag.

In [ ]:
# Export a configured Scenario to my_exported_scenario.xlsx
!atmn-export my_collection Hanoi_One_Week MultipleLeaks Sensors1 PressureAndFlow my_exported_scenario

## Visualizing
When creating Scenario configs, it proves useful to have a visualization of the water network. To facilitate this, we also provide the `atmn-visualize` tool.

If you only pass the script an `inp` file, it will just plainly plot it. If you also want to get an overview of leaks, sensors and sensorfaults, you can pass it your config file. If you decide to do so, you also need to specify which scenario to display, as well as which leak, sensor and sensorfault configs.

The visualization is color-coded:
* Leaks: Blue
* Sensorfaults: Red
* Sensors: Green

*The upper colors take precedence. So if for example a part contains a leak and a sensor, it will be blue.*

You can also specify an output file using the `--output` argument. The visualization will be saved in this file in html format. If no output file is specified, a temporary file is created and automatically opened in your browser.

In [ ]:
# Create a visualization of the Hanoi network
!atmn-visualize networks/Hanoi.inp -o Hanoi_Network.html

# Create a visualization including Leaks, Sensors and Sensorfaults
!atmn-visualize my_collection/config.xml Hanoi_One_Week MultipleLeaks Sensors1 PressureAndFlow -o Hanoi_Scenario.html